In [41]:
# -*- coding: utf-8 -*-
"""
20240202
自有有邮件通知系统整合到quantlab3.5内，实现模型计算回测，数据下载，邮件通知一体化运行
同时保留本地调试，策略研发
使用GitHub进行代码备份
"""
# ------ 导入包-----   ##
import akshare as ak
import pandas as pd
import numpy as np
import datetime as dt
import math
from dateutil.relativedelta import relativedelta
import os
import subprocess


In [15]:
# 获取股票指数数据
def get_index_data(symbol="000016", period="daily", start_date="20240401", end_date="20240531"):
    index_data =ak.index_zh_a_hist(symbol,period,start_date,end_date)
    return index_data


In [16]:
start_date ='20240501'
end_date = '20240531'
df = get_index_data()

In [17]:
df.head(3)

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2024-04-01,2422.73,2441.57,2448.88,2422.73,36210013,6.316180e+10,1.08,1.10,26.65,0.23
1,2024-04-02,2441.49,2435.76,2447.37,2428.03,35726428,5.991735e+10,0.79,-0.24,-5.81,0.22
2,2024-04-03,2439.43,2430.17,2441.67,2423.88,31959986,6.009749e+10,0.73,-0.23,-5.59,0.20


In [18]:
def calculate_statistics(data):
    data["涨跌幅"] = (data["收盘"].pct_change() * 100).round(2).astype(str) + '%'
    data["近30日均值"] = data["收盘"].rolling(window=30).mean()

    # 计算boll轨 MA20 P= 2
    data["近20日均值"] = data["收盘"].rolling(window=20).mean()
    data['20日移动标准差'] = data['收盘'].rolling(window=20).std()
    # 计算布林轨区间
    data['布林轨下轨'] = data['近20日均值'] - 1.3 * data['20日移动标准差']
    data['布林轨上轨'] = data['近20日均值'] + 1.3 * data['20日移动标准差']
    data['布林轨区间'] = list(zip((data['布林轨下轨']).round(2), (data['布林轨上轨']).round(2)))
    # 操作提示
    data["预警"] = data.apply(
        lambda row: "突破上轨" if row["收盘"] > row['布林轨上轨'] else "突破下轨" if row["收盘"] < row[
            '布林轨下轨'] else '震荡', axis=1)

    return data

In [20]:
df1 = calculate_statistics(df)
df1.head(2)

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,近30日均值,近20日均值,20日移动标准差,布林轨下轨,布林轨上轨,布林轨区间,预警
0,2024-04-01,2422.73,2441.57,2448.88,2422.73,36210013,6.316180e+10,1.08,nan%,26.65,0.23,NaN,NaN,NaN,NaN,NaN,"(nan, nan)",震荡
1,2024-04-02,2441.49,2435.76,2447.37,2428.03,35726428,5.991735e+10,0.79,-0.24%,-5.81,0.22,NaN,NaN,NaN,NaN,NaN,"(nan, nan)",震荡


In [21]:
# 获取观察指数的统计信息
def get_observation_statistics(index_name, index_code, data):
    latest_data = data.iloc[-1]
    statistics = {
        "观察指数": index_name,
        "指数代码": index_code,
        "当日数值": latest_data['收盘'],
        "涨跌幅": latest_data['涨跌幅'],
        "昨日数值": data.iloc[-2]['收盘'],
        "前日数值": data.iloc[-3]['收盘'],
        "近30日均值": latest_data['近30日均值'],
        "布林轨区间": latest_data['布林轨区间'],
        "预警": latest_data['预警']
    }
    return statistics

In [ ]:
df2 = get_observation_statistics()

In [68]:
import akshare as ak
import pandas as pd

class IndexAnalyzer:
    def __init__(self, period, start_date, end_date, index_codes, index_names):
        self.period = period
        self.start_date = start_date
        self.end_date = end_date
        self.index_codes = index_codes
        self.index_names = index_names

    def get_index_data(self, symbol):
        index_data = ak.index_zh_a_hist(symbol, self.period, self.start_date, self.end_date)
        return index_data

    def calculate_statistics(self, data):
        if data.empty or len(data) < 2:
            return data

        data["涨跌幅"] = (data["收盘"].pct_change() * 100).round(2).astype(str) + '%'
        data["近30日均值"] = data["收盘"].rolling(window=30).mean()

        # 计算boll轨 MA20 P= 2
        data["近20日均值"] = data["收盘"].rolling(window=20).mean()
        data['20日移动标准差'] = data['收盘'].rolling(window=20).std()
        # 计算布林轨区间
        data['布林轨下轨'] = data['近20日均值'] - 1.3 * data['20日移动标准差']
        data['布林轨上轨'] = data['近20日均值'] + 1.3 * data['20日移动标准差']
        data['布林轨区间'] = list(zip((data['布林轨下轨']).round(2), (data['布林轨上轨']).round(2)))
        # 操作提示
        data["预警"] = data.apply(
            lambda row: "突破上轨" if row["收盘"] > row['布林轨上轨'] else "突破下轨" if row["收盘"] < row[
                '布林轨下轨'] else '震荡', axis=1)

        return data

    def get_observation_statistics(self, index_name, index_code, data):
        if data.empty or len(data) < 3:
            return {}

        latest_data = data.iloc[-1]
        statistics = {
            "观察日期": data.iloc[-1]['日期'],
            "观察指数": index_name,
            "指数代码": index_code,
            "当日数值": latest_data['收盘'],
            "涨跌幅": latest_data['涨跌幅'],
            "昨日数值": data.iloc[-2]['收盘'],
            "前日数值": data.iloc[-3]['收盘'],
            "近30日均值": latest_data['近30日均值'],
            "布林轨区间": latest_data['布林轨区间'],
            "预警": latest_data['预警']
        }
        return statistics

    def get_index_statistics(self, start_date, end_date):
        # 初始化结果 DataFrame
        result_df = pd.DataFrame(
            columns=["观察日期","观察指数", "指数代码", "当日数值", "涨跌幅", "昨日数值", "前日数值", "近30日均值", "布林轨区间",
                     "预警"])

        # 获取指数数据并计算统计信息
        for index_name, index_code in zip(self.index_names, self.index_codes):
            index_data = self.get_index_data(index_code)
            index_data = self.calculate_statistics(index_data)
            statistics = self.get_observation_statistics(index_name, index_code, index_data)
            # 将字典转换为DataFrame
            if statistics:
                statistics_df = pd.DataFrame([statistics])
                # 将statistics_df添加到result_df
                result_df = pd.concat([result_df, statistics_df], ignore_index=True)

        return result_df



In [69]:
# 示例使用
from datetime import datetime, timedelta
index_codes = ["000001", "399001", "399006", "000688"]
index_names = ["上证指数", "深圳指数", "创业板", "科创板"]
start_date = (datetime.now() - timedelta(days=60)).strftime("%Y%m%d")
end_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
analyzer = IndexAnalyzer("daily", start_date, end_date, index_codes, index_names)
result = analyzer.get_index_statistics(start_date, end_date)
result.to_csv("data/index_statistics.csv")

C:\Users\00\AppData\Local\Temp\ipykernel_31364\1264064334.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, statistics_df], ignore_index=True)


In [70]:
result

,观察日期,观察指数,指数代码,当日数值,涨跌幅,昨日数值,前日数值,近30日均值,布林轨区间,预警
0,2024-07-04,上证指数,000001,2949.93,-0.26%,2957.57,2982.38,3027.416333,"(2952.86, 3039.8)",突破下轨
1,2024-07-04,深圳指数,399001,8695.55,0.25%,8673.83,8760.43,9152.055000,"(8739.96, 9310.12)",突破下轨
2,2024-07-04,创业板,399006,1655.59,0.51%,1647.22,1660.12,1763.439333,"(1662.62, 1807.48)",突破下轨
3,2024-07-04,科创板,000688,698.85,1.23%,690.38,701.78,735.838667,"(700.8, 764.69)",突破下轨


In [64]:
result

,观察指数,指数代码,当日数值,涨跌幅,昨日数值,前日数值,近30日均值,布林轨区间,预警
0,上证指数,000001,2949.93,-0.26%,2957.57,2982.38,3027.416333,"(2952.86, 3039.8)",突破下轨
1,深圳指数,399001,8695.55,0.25%,8673.83,8760.43,9152.055000,"(8739.96, 9310.12)",突破下轨
2,创业板,399006,1655.59,0.51%,1647.22,1660.12,1763.439333,"(1662.62, 1807.48)",突破下轨
3,科创板,000688,698.85,1.23%,690.38,701.78,735.838667,"(700.8, 764.69)",突破下轨


In [54]:
# 示例使用
index_codes = ["000001", "399001", "399006", "000688"]
index_names = ["上证指数", "深圳指数", "创业板", "科创板"]
analyzer = IndexAnalyzer("daily", "20240101", "20240531", index_codes, index_names)
result = analyzer.get_index_statistics("20240101", "20240531")
print(result)

C:\Users\00\AppData\Local\Temp\ipykernel_31364\2937429974.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, statistics_df], ignore_index=True)


   观察指数    指数代码     当日数值     涨跌幅     昨日数值     前日数值       近30日均值  \
0  上证指数  000001  3086.81  -0.16%  3091.68  3111.02  3110.454333   
1  深圳指数  399001  9364.38  -0.22%  9384.72  9414.98  9526.759000   
2   创业板  399006  1805.11  -0.44%  1813.15  1811.07  1833.060667   
3   科创板  000688   743.23   0.07%   742.70   734.33   750.141667   

                布林轨区间    预警  
0  (3098.66, 3164.54)  突破下轨  
1   (9417.0, 9793.21)  突破下轨  
2   (1812.9, 1891.86)  突破下轨  
3    (733.75, 771.21)    震荡  
